In [1]:
#Load the MNIST dasaset
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

/home/sindre/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
#Start TensorFlow InteractiveSession
import tensorflow as tf
sess = tf.InteractiveSession()

## Building Softmax Regression Model

In [3]:
#Start building of the computation graph by creating nodes for the input images and target output classes
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

Placeholder = value that will be defined when TensorFlow is asked to run a computation <br>
x = input images consisting of a 2d tensor of floating point numbers. <br>
Give x a shape of [None, 784] because 784 is the dimensionality of a single flattened MNIST image (28x28=784) <br>
None -> indicates that the first dimension, corresponding to the batch size, can be of any size <br>
y_ (target output classes) -> consists of a 2d tensor, 
    where each row is a one-hot 10-dimensional vector indicating which digit class (zero through nine) the
    corresponding MNIST image belongs to <br>
Shape argument is optional, but allows TensorFlow to automatically catch bugs stemming from inconsistent tensor shapes
    

## Variables

In [4]:
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

W = 784x10 matrix --> 784 input features and 10 outputs
<br> b = 10-dimensional vector (10 classes)

In [5]:
#Initialize the variables within the session, in order to be utilized 
sess.run(tf.global_variables_initializer())

## Predicted Class and Loss Function

In [6]:
#Implementing the regression model
#Multiplying the vectorized input images x by the weight matrix W, add the bias b
y = tf.matmul(x,W) + b

In [7]:
#Specify the loss function, cross-entropy (stable formulation)
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

## Training the model

In [8]:
#Training the model using Steepest Gradient Descent, with a step length of 0.5, to descend the cross entropy
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

This adds new operations to the computation graph, including
     1. ones to compute gradients
     2. computer parameter update steps
     3. Apply update steps to the parameters

When train_steps will apply the gradient descent updates to the parameters
Traning the model is therefore accomplished by repeatedly running train_steps

In [9]:
for _ in range(1000):
    batch = mnist.train.next_batch(100)
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})

## Evaluating the Model

tf.argmax gives the index of the highest entry in a tensor along some axis <br>
Example: <br>
tf.argmax(y, 1) -> the label the model thinks is most likely for each input <br>
tf.argmax(y_, 1) -> true label <br>
Uses tf.equal to check if prediction matches the truth <br>

In [10]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))

Returns a list of booleans. <br>
Determine what fraction are correct: <br>
1. Cast floating point numbers
2. Take the mean <br>
Example: <br>
[True, False, True, True] -> [1, 0, 1, 1] -> 0.75

In [11]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [12]:
#Evaluating the accuracy
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9176


## Improving the model - Building a MultiLayer Convolutional Network

### Weight Initialization
Need to create weights and biases in order to create the model. <br>
Generally initializes weights with a small amount of noise -> symmetri breaking, and preventing 0 gradients <br>
Creating to handy functions:

In [13]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

### Convolution and Pooling
Convolution uses a stride of one and are zero padded so that the output is the same size as the input <br>
Pooling is plain old max pooling over 2x2 blocks <br>
Abstracting those operations into functions --> cleaner code

In [14]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

### First Convolutional Layer
The first layer will consist of convolution, followed by max pooling <br>
The convolution computes 32 features for each 5x5 patch, with a weight tensor shaped like [5, 5, 1, 32]. <br>
First 2 dimensions = patch size, next is number of input channels, and last is number of output channels. <br>
Will also have a bias vector with a component for each output channel

In [15]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

In [16]:
#Applying the first layer, need to reshape x to a 4d tensor, with the second and third dimensions corresponding to
#image width and height, and the final dimension corresponding to the number of color channels
x_image = tf.reshape(x, [-1, 28, 28, 1])

In [17]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

### Second Convolutional Layer
To build a deep network -> stack several layers of this type. <br>
Second layer will have 64 features for each 5x5 patch

In [18]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

### Densely Connected Layer
Images size has now been reduced to 7x7 -> add a fully-connected layer with 1024 neurons to allow processing on the 
entire image. <br>
Reshape the tensor from the pooling layer into a batch of vectors, multiply by a weight matrix, add a bias, and
apply a ReLU

In [19]:
W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat=tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

### Dropout
Apply *Dropout* before the redout layer, in order to reduce overfitting. <br>
Create a *Placeholder* for the probability that a neuron's output is kept during dropout. <br>
--> Turn dropout in during training, and turn it off during testing. <br>
TF's tf.nn.dropout op automatically handles scaling neuron outputs in addition to masking them, so dropout just works without any additional scaling <br>

Dropout is often very effective at reducing overfitting, but it is most useful when training very large neural networks. 

In [20]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

### Readout Layer

In [21]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

### Train and Evaluate the Model
Instead of using steepest gradient descent optimizer --> ADAM optimizer <br>
Include the additional parameter *keep_prob* in *feed_dict* to control dropout rate <br>
Add logging to every 100th iteration in the training process <br>

Uses tf.Session rather than tf.InteractiveSession: <br>
1. Better seperates the process of creating the graph (model specification) and process of evaluating the graph (model fitting).
2. Makes for a cleaner code <br>
The tf.Session is created with a *with* block, so that it is automatically destroyed once the block is exited. 

In [23]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i % 100 == 0:
      train_accuracy = accuracy.eval(feed_dict={
          x: batch[0], y_: batch[1], keep_prob: 1.0})
      print('step %d, training accuracy %g' % (i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

  print('test accuracy %g' % accuracy.eval(feed_dict={
      x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.18
step 100, training accuracy 0.82
step 200, training accuracy 0.86
step 300, training accuracy 0.96
step 400, training accuracy 0.86
step 500, training accuracy 0.94
step 600, training accuracy 0.98
step 700, training accuracy 1
step 800, training accuracy 0.92
step 900, training accuracy 0.98
step 1000, training accuracy 0.94
step 1100, training accuracy 0.9
step 1200, training accuracy 0.92
step 1300, training accuracy 0.96
step 1400, training accuracy 1
step 1500, training accuracy 0.98
step 1600, training accuracy 0.98
step 1700, training accuracy 1
step 1800, training accuracy 1
step 1900, training accuracy 0.92
step 2000, training accuracy 1
step 2100, training accuracy 0.98
step 2200, training accuracy 0.98
step 2300, training accuracy 0.98
step 2400, training accuracy 0.96
step 2500, training accuracy 1
step 2600, training accuracy 0.98
step 2700, training accuracy 0.98
step 2800, training accuracy 0.98
step 2900, training accuracy 0.98
step 3000, 